In [ ]:
import requests
import pandas as pd

In [ ]:
fd = pd.read_csv("CVE_list.csv",names=['cve_id'], )

In [ ]:
cve_list = list(fd.cve_id)
cve_list[:10]

***Algorithm:***<br>
For each CVE_ID in the list:<br>
&emsp; Do a get request from https://services.nvd.nist.gov/rest/json/cve/1.0/<CVE-ID> API
<br>&emsp; Read the JSON output to a Python Dictonary data type variable
<br>&emsp; If the response from request is not NULL
<br>&emsp; &emsp; Get the individual values. (For Suse Linux baseMetricV2_baseScore and baseMetricV2_severity are used instead of cvssV3 metrics.)
<br> &emsp; &emsp; append to a list
<br> &emsp; Set all variables to NULL before looping to next CVE_ID

***Coding Tip :*** Instead of creating a empty dataframe and appending rows, better to build list of list and create a dataframe at the end. this is more optimal on memory utilization. <dr> https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.append.html <dr>
* Iteratively appending rows to a DataFrame can be more computationally intensive than a single concatenate. A better solution is to append those rows to a list and then concatenate the list with the original DataFrame all at once.

In [ ]:
# Create a list of columns that need to be heading in CSV file
cols=['cve_id','cvssV3_baseScore','cvssV3_baseSeverity','cvssV3_vectorString','description','publishedDate','lastModifiedDate','baseMetricV2_baseScore','baseMetricV2_severity']
data = []
for cve in cve_list:
    req="https://services.nvd.nist.gov/rest/json/cve/1.0/"+cve
    response = requests.get(req)
    if response.json().get('result') is not None:
        publishedDate = response.json().get('result').get('CVE_Items')[0].get('publishedDate', ' ')
        lastModifiedDate = response.json().get('result').get('CVE_Items')[0].get('lastModifiedDate', ' ')
        description = response.json().get('result').get('CVE_Items')[0].get('cve').get('description').get('description_data')[0].get('value',' ')
        baseMetricV3 = response.json().get('result').get('CVE_Items')[0].get('impact').get('baseMetricV3')
        if baseMetricV3 is not None:
            cvssV3_baseScore = response.json().get('result').get('CVE_Items')[0].get('impact').get('baseMetricV3').get('cvssV3').get('baseScore',' ')
            cvssV3_baseSeverity = response.json().get('result').get('CVE_Items')[0].get('impact').get('baseMetricV3').get('cvssV3').get('baseSeverity',' ')
            cvssV3_vectorString = response.json().get('result').get('CVE_Items')[0].get('impact').get('baseMetricV3').get('cvssV3').get('vectorString',' ')
        baseMetricV2 = response.json().get('result').get('CVE_Items')[0].get('impact').get('baseMetricV2')
        if baseMetricV2 is not None:
            baseMetricV2_baseScore = response.json().get('result').get('CVE_Items')[0].get('impact').get('baseMetricV2').get('cvssV2').get('baseScore',' ')
            baseMetricV2_severity = response.json().get('result').get('CVE_Items')[0].get('impact').get('baseMetricV2').get('severity',' ')
        #print(cve)
        data.append([cve,cvssV3_baseScore,cvssV3_baseSeverity,cvssV3_vectorString,description,publishedDate,lastModifiedDate,baseMetricV2_baseScore,baseMetricV2_severity])
        cvssV3_baseScore=cvssV3_baseSeverity=cvssV3_vectorString=description=publishedDate = lastModifiedDate = baseMetricV2_baseScore = baseMetricV2_severity = None

In [ ]:
output.to_csv('vulnerability_report.csv', sep=',', encoding='utf-8')